### Beginning of the Assignment - exploration

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
#pd.set_option('max_columns', 200)

### PHIL DATA SET

In [2]:
character_nicknames_df = pd.read_csv('datasets/PHIL DATA SET/character_nicknames.csv')

In [13]:
details_df = pd.read_csv('datasets/PHIL DATA SET/details.csv')

In [ ]:
favs_df = pd.read_csv('datasets/PHIL DATA SET/favs.csv')


In [46]:
person_details_df = pd.read_csv('datasets/PHIL DATA SET/person_details.csv')


In [ ]:
person_alternate_names_df = pd.read_csv('datasets/PHIL DATA SET/person_alternate_names.csv')


In [ ]:
person_anime_works_df = pd.read_csv('datasets/PHIL DATA SET/person_anime_works.csv')


In [ ]:
stats_df = pd.read_csv('datasets/PHIL DATA SET/stats.csv')


### DENIS DATA SET

In [ ]:
ratings_df = pd.read_csv('datasets/DENIS DATA SET/ratings.csv')

In [20]:
characters_df = pd.read_csv('datasets/DENIS DATA SET/characters.csv')

In [36]:
character_anime_works_df = pd.read_csv('datasets/DENIS DATA SET/character_anime_works.csv')

In [42]:
person_voice_works_df = pd.read_csv('datasets/DENIS DATA SET/person_voice_works.csv')

In [57]:
profiles_df = pd.read_csv('datasets/DENIS DATA SET/profiles.csv')

In [71]:
recommendations_df = pd.read_csv('datasets/DENIS DATA SET/recommendations.csv')

##### First look

In [35]:
character_nicknames_df.shape

(37080, 2)

In [ ]:
character_nicknames_df.head()

In [ ]:
character_nicknames_df.columns
#will list all the columns. Not necessary here but kept as a pattern to follow with the following files

In [ ]:
character_nicknames_df.dtypes

In [ ]:
character_nicknames_df.describe()
#not needed here because it should just be ids

### Data preparation (cleaning)


##### On the first dataset we may need to check for duplicates.
What does that mean? We have 102 rows that are duplicated over a 37080 rows dataset.
Why is that? Are there some characters that have multiple nicknames, so they are repeated in the dataset?

In [ ]:
character_nicknames_df.loc[character_nicknames_df.duplicated()]
#by default will give us the second

In [ ]:
character_nicknames_df.query('character_mal_id == 75628')

Mhh they're somehow different so yeah, the same character could have different nicknames.
We want to drop though the ones that are exactly the same.

In [ ]:
#this way we drop the duplicates on the first dataset

character_nicknames_df = character_nicknames_df\
    .loc[~character_nicknames_df.duplicated()]\
    .reset_index(drop=True).copy()

#we don't need to use a subset here because there are just 2 columns

Let's check for nan values

In [ ]:
character_nicknames_df[character_nicknames_df.isna().any(axis=1)]


In [ ]:
#cleaning the df from nan values
character_nicknames_df = character_nicknames_df.dropna()


## SECOND DATASET

##### On the second dataset we may need to check for missing values and/or inconsistent values, since there are no duplicates

In [ ]:
details_df.isna().sum()

In [14]:
details_df
#anime details
#japanes title could be dropped?
#members stand for how many users have this anime added to their list.
#explicit_genres is empty so can be removed
#licensor and streaming are mostly empty. Do we care?

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,demographics,source,rating,episodes,season,year,producers,explicit_genres,licensors,streaming
0,59356,-Socket-,-socket-,https://myanimelist.net/anime/59356/-Socket-,https://cdn.myanimelist.net/images/anime/1043/...,Movie,Finished Airing,NaN,NaN,2010-01-01T00:00:00+00:00,...,[],Original,G - All Ages,1.0,NaN,NaN,['Nagoya Zokei University'],[],[],[]
1,56036,......,......,https://myanimelist.net/anime/56036/-,https://cdn.myanimelist.net/images/anime/1057/...,Music,Finished Airing,6.53,503.0,2023-06-11T00:00:00+00:00,...,[],Original,PG-13 - Teens 13 or older,1.0,NaN,NaN,[],[],[],[]
2,2928,.hack//G.U. Returner,.HACK//G.U. RETURNER,https://myanimelist.net/anime/2928/hack__GU_Re...,https://cdn.myanimelist.net/images/anime/1798/...,OVA,Finished Airing,6.65,9745.0,2007-01-18T00:00:00+00:00,...,[],Game,PG-13 - Teens 13 or older,1.0,NaN,NaN,"['Bandai Visual', 'CyberConnect2']",[],[],[]
3,3269,.hack//G.U. Trilogy,.hack//G.U. Trilogy,https://myanimelist.net/anime/3269/hack__GU_Tr...,https://cdn.myanimelist.net/images/anime/1566/...,Movie,Finished Airing,7.06,15373.0,2007-12-22T00:00:00+00:00,...,[],Game,PG-13 - Teens 13 or older,1.0,NaN,NaN,['Bandai Visual'],[],"['Funimation', 'Bandai Entertainment']",[]
4,4469,.hack//G.U. Trilogy: Parody Mode,.hack//G.U. Trilogy,https://myanimelist.net/anime/4469/hack__GU_Tr...,https://cdn.myanimelist.net/images/anime/10/86...,Special,Finished Airing,6.35,4317.0,2008-03-25T00:00:00+00:00,...,[],Game,PG-13 - Teens 13 or older,1.0,NaN,NaN,['Bandai Visual'],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28950,59421,Zutaboro Reijou wa Ane no Moto Konyakusha ni D...,ずたぼろ令嬢は姉の元婚約者に溺愛される,https://myanimelist.net/anime/59421/Zutaboro_R...,https://cdn.myanimelist.net/images/anime/1518/...,TV,Finished Airing,7.37,15624.0,2025-07-05T00:00:00+00:00,...,['Josei'],Light novel,PG-13 - Teens 13 or older,12.0,summer,2025.0,"['Studio Pierrot', 'Mainichi Broadcasting Syst...",[],[],"['Crunchyroll', 'Aniplus TV', 'Bahamut Anime C..."
28951,31245,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,ずっと前から好きでした。～告白実行委員会～,https://myanimelist.net/anime/31245/Zutto_Mae_...,https://cdn.myanimelist.net/images/anime/3/821...,Movie,Finished Airing,7.20,104106.0,2016-04-23T00:00:00+00:00,...,[],Music,PG-13 - Teens 13 or older,1.0,NaN,NaN,"['Aniplex', 'Dentsu', 'Kadokawa Shoten', 'Movi...",[],['Aniplex of America'],[]
28952,36305,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,ずっと前から好きでした。～告白実行委員会～ 「金曜日のおはよう」,https://myanimelist.net/anime/36305/Zutto_Mae_...,https://cdn.myanimelist.net/images/anime/6/883...,Special,Finished Airing,7.17,10038.0,2016-10-26T00:00:00+00:00,...,[],Music,PG - Children,1.0,NaN,NaN,['Aniplex'],[],[],[]
28953,34895,Zutto Suki Datta,ずっと好きだった,https://myanimelist.net/anime/34895/Zutto_Suki...,https://cdn.myanimelist.net/images/anime/1498/...,OVA,Finished Airing,5.68,1887.0,2017-04-21T00:00:00+00:00,...,[],Manga,Rx - Hentai,2.0,NaN,NaN,"['Queen Bee', 'Mediabank']",[],[],[]


In [ ]:
details_df[['start_date','season']].query("season.notna()")
#season can be removed? Do we care about the season? We can "calculate" it from the "start_date" field

In [ ]:
details_df.query("episodes > 2500")

In [ ]:
details_df.dtypes
#scoredby, rank, episodes, year can be an int instead of a float
#start and end dates are not objects but dates
#do we need to swap the empty [] with Nan or not? WE should in order to be able to use the .isna() method and other pandas methods


### THIRD DATASET

In [ ]:
favs_df

In [ ]:
favs_df.isna().sum()


In [ ]:
favs_df.dtypes

### FOURTH DATASET

In [ ]:
person_alternate_names_df

In [ ]:
person_alternate_names_df.dtypes

In [ ]:
person_alternate_names_df.isna().sum()

In [ ]:
person_alternate_names_df = person_alternate_names_df.dropna()


### FIFTH DATASET

In [ ]:
#let's check for duplicates

In [47]:
person_details_df

,person_mal_id,url,website_url,image_url,name,given_name,family_name,birthday,favorites,relevant_location
0,1,https://myanimelist.net/people/1/Tomokazu_Seki,NaN,https://cdn.myanimelist.net/images/voiceactors...,Tomokazu Seki,智一,関,1972-09-08T00:00:00+00:00,6219,"Berlin, Germany"
1,2,https://myanimelist.net/people/2/Tomokazu_Sugita,https://agrs.co.jp/,https://cdn.myanimelist.net/images/voiceactors...,Tomokazu Sugita,智和,杉田,1980-10-11T00:00:00+00:00,47666,"Los Angeles, USA"
2,3,https://myanimelist.net/people/3/Satsuki_Yukino,NaN,https://cdn.myanimelist.net/images/voiceactors...,Satsuki Yukino,さつき,ゆきの,1970-05-25T00:00:00+00:00,1777,"Madrid, Spain"
3,4,https://myanimelist.net/people/4/Aya_Hirano,http://ayahirano.jp/,https://cdn.myanimelist.net/images/voiceactors...,Aya Hirano,綾,平野,1987-10-08T00:00:00+00:00,18374,"Paris, France"
4,5,https://myanimelist.net/people/5/Kenichi_Suzumura,https://intention-k.com,https://cdn.myanimelist.net/images/voiceactors...,Kenichi Suzumura,健一,鈴村,1974-09-12T00:00:00+00:00,5176,"Osaka, Japan"
...,...,...,...,...,...,...,...,...,...,...
76694,90011,https://myanimelist.net/people/90011/Nanako_Ki...,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,Nanako Kishimoto,七子,岸本,NaN,0,"Mumbai, India"
76695,90012,https://myanimelist.net/people/90012/Pamon,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,Pamon,NaN,파몬,NaN,0,"Tokyo, Japan"
76696,90013,https://myanimelist.net/people/90013/Tomoru_Emoto,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,Tomoru Emoto,ともる,柄本,NaN,0,"Tokyo, Japan"
76697,90014,https://myanimelist.net/people/90014/Hirari,https://hirari.2-d.jp/,https://cdn.myanimelist.net/img/sp/icon/apple-...,Hirari,NaN,ひらり,NaN,0,"Paris, France"


In [ ]:
person_details_df.loc[person_details_df['person_mal_id'].duplicated()]
#we found that the duplicates differ for the "relevant_location" field, which has no interest for us so we drop the duplicates
#TODO

In [ ]:
person_details_df[person_details_df.isna().any(axis=1)]


In [ ]:
person_details_df.dtypes
#we need to change birthday from object to data

In [66]:
person_details_df.isna().sum()
#we have to check the nan values

person_mal_id            0
url                      0
website_url          59264
image_url                0
name                     2
given_name           30253
family_name          18641
birthday             59461
favorites                0
relevant_location        0
dtype: int64

We can join the two tables person_details_df and person_alternate_names_df having the keys that match.
Putting the alternate names in a new column called alt_name and having a list of those inside

### SIXTH DATASET

In [ ]:
person_anime_works_df

In [ ]:
person_anime_works_df.dtypes
#the types are correct

In [ ]:
person_anime_works_df.isna().sum()
#There's no nan value

### SEVENTH DATASET

In [ ]:
stats_df

In [ ]:
stats_df.dtypes
#we can swap *_votes to int instead of floats

In [ ]:
stats_df.isna().sum()
#there are 430 series without any vote (?)
#we need to understand which ones are they

In [ ]:
stats_df[stats_df['score_1_votes'].isna()]


In [ ]:
details_df.query('mal_id == 61558')

### EIGHTH DATASET 

In [8]:
ratings_df

,username,anime_id,status,score,is_rewatching,num_watched_episodes
0,--------788,30276,watching,7,0.0,3
1,--------788,28851,completed,7,0.0,1
2,--------788,41168,completed,7,0.0,1
3,--------788,22199,completed,10,0.0,24
4,--------788,16498,completed,10,0.0,25
...,...,...,...,...,...,...
124298352,arizkim,52305,plan_to_watch,0,0.0,0
124298353,arizkim,4224,plan_to_watch,0,0.0,0
124298354,arizkim,54790,plan_to_watch,0,0.0,0
124298355,arizkim,53835,plan_to_watch,0,0.0,0


In [9]:
# check "num_watched_episodes"
ratings_df.query('is_rewatching == 1')

,username,anime_id,status,score,is_rewatching,num_watched_episodes
934,CKK2,6,completed,9,1.0,8
8939,FollowYourHeart,1210,completed,10,1.0,24
11047,----Haku----,39017,completed,7,1.0,3
22879,KarioBaka,10087,completed,9,1.0,3
22928,KarioBaka,14467,completed,0,1.0,6
...,...,...,...,...,...,...
124291819,arissabelle,2167,completed,0,1.0,0
124291878,arissabelle,33352,completed,10,1.0,0
124291883,arissabelle,6547,on_hold,0,1.0,1
124297299,ariyanroy04,46569,completed,0,1.0,13


In [17]:
details_df.query('mal_id == 918')

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,demographics,source,rating,episodes,season,year,producers,explicit_genres,licensors,streaming
7358,918,Gintama,銀魂,https://myanimelist.net/anime/918/Gintama,https://cdn.myanimelist.net/images/anime/10/73...,TV,Finished Airing,8.93,415923.0,2006-04-04T00:00:00+00:00,...,['Shounen'],Manga,PG-13 - Teens 13 or older,201.0,spring,2006.0,"['TV Tokyo', 'Aniplex', 'Dentsu', 'Trinity Sou...",[],"['Sentai Filmworks', 'Crunchyroll']","['Crunchyroll', 'Netflix']"


In [10]:
# change "is_rewatching" from float to int or boolean
ratings_df.dtypes

username                 object
anime_id                  int64
status                   object
score                     int64
is_rewatching           float64
num_watched_episodes      int64
dtype: object

In [12]:
ratings_df.loc[ratings_df.duplicated(subset=['username', 'anime_id'])]

,username,anime_id,status,score,is_rewatching,num_watched_episodes
44469797,W00F1234,59457,dropped,3,0.0,1
74614909,Pavle2009,44511,watching,0,0.0,7
74616569,Doopelsdoo,59459,watching,0,0.0,9
74626628,Door_mp3,59644,watching,8,0.0,6
74626629,Door_mp3,60564,watching,8,0.0,5
74626630,Door_mp3,61159,watching,9,0.0,5


In [18]:
# we check for Nan values. 
# if it necessary check if the num_watched_episodes is greater than number of episodes of anime, we can remove the Nan values and put one or zero. 
ratings_df.isna().sum()

username                      7
anime_id                      0
status                        0
score                         0
is_rewatching           3797321
num_watched_episodes          0
dtype: int64

In [19]:
# drop "username" with Nan values
ratings_df[ratings_df['username'].isna()]

,username,anime_id,status,score,is_rewatching,num_watched_episodes
100051758,NaN,1482,watching,9,0.0,35
100051759,NaN,1735,watching,9,0.0,21
100051760,NaN,121,completed,10,0.0,51
100051761,NaN,136,completed,8,0.0,62
100051762,NaN,269,on_hold,7,0.0,33
100051763,NaN,1818,on_hold,7,0.0,11
100051764,NaN,1535,plan_to_watch,6,0.0,4


### NINTH DATASET

In [21]:
characters_df

,character_mal_id,url,name,name_kanji,image,favorites,about
0,280386.0,https://myanimelist.net/character/280386/Envi_...,Envi Mel Champagne,エンヴィ・メル・シャンパーニュ,https://cdn.myanimelist.net/images/characters/...,0.0,NaN
1,280354.0,https://myanimelist.net/character/280354/Eleven,Eleven,イレヴン,https://cdn.myanimelist.net/images/characters/...,0.0,NaN
2,280353.0,https://myanimelist.net/character/280353/Stud,Stud,スタッド,https://cdn.myanimelist.net/images/characters/...,0.0,NaN
3,280352.0,https://myanimelist.net/character/280352/Judge,Judge,ジャッジ,https://cdn.myanimelist.net/images/characters/...,0.0,NaN
4,280339.0,https://myanimelist.net/character/280339/Eiji_...,Eiji Kurokawa,黒川 英治,https://cdn.myanimelist.net/img/sp/icon/apple-...,0.0,NaN
...,...,...,...,...,...,...,...
209958,282276.0,https://myanimelist.net/character/282276/Farra...,Farrah Van Dorothy,ファラ・ヴァン・ドロシー,https://cdn.myanimelist.net/images/characters/...,0.0,NaN
209959,282277.0,https://myanimelist.net/character/282277/Harri...,Harris Mead,ハリス・ミード,https://cdn.myanimelist.net/images/characters/...,0.0,NaN
209960,282278.0,https://myanimelist.net/character/282278/Rob,Rob,ロブ,https://cdn.myanimelist.net/images/characters/...,0.0,NaN
209961,282281.0,https://myanimelist.net/character/282281/Grimm,Grimm,グリム,https://cdn.myanimelist.net/images/characters/...,0.0,NaN


In [22]:
# change "character_mal_id" and "favorites" from float to int
characters_df.dtypes

character_mal_id    float64
url                  object
name                 object
name_kanji           object
image                object
favorites           float64
about                object
dtype: object

In [25]:
characters_df.query('favorites == 175632.000000')

,character_mal_id,url,name,name_kanji,image,favorites,about
208315,417.0,https://myanimelist.net/character/417/Lelouch_...,Lelouch Lamperouge,ルルーシュ・ランペルージ,https://cdn.myanimelist.net/images/characters/...,175632.0,"Age: 17 (first season), 18 (second season)\nBi..."


In [24]:
characters_df.describe()

,character_mal_id,favorites
count,209961.000000,209961.000000
mean,150579.010283,57.616138
std,85243.624423,1197.586488
min,1.000000,0.000000
25%,74581.000000,0.000000
50%,160191.000000,0.000000
75%,225373.000000,2.000000
max,282284.000000,175632.000000


In [26]:
characters_df.isna().sum()

character_mal_id        2
url                     2
name                    2
name_kanji          55480
image                   2
favorites               2
about               96976
dtype: int64

In [28]:
# drop the two rows with all columns Nan
characters_df[characters_df['character_mal_id'].isna()]

,character_mal_id,url,name,name_kanji,image,favorites,about
209733,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209734,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# drop duplicates 
characters_df.loc[characters_df.duplicated(keep="first")]

,character_mal_id,url,name,name_kanji,image,favorites,about
854,279200.0,https://myanimelist.net/character/279200/Shing...,Shingo Shimazaki,島崎 真悟,https://cdn.myanimelist.net/images/characters/...,1.0,NaN
910,279142.0,https://myanimelist.net/character/279142/Kaho,Kaho,NaN,https://cdn.myanimelist.net/images/characters/...,0.0,NaN
1167,278865.0,https://myanimelist.net/character/278865/Yoshi...,Yoshirou Sonozaki,園崎 義郎,https://cdn.myanimelist.net/images/characters/...,0.0,The owner of the restaurant Angel Mort.
1281,278746.0,https://myanimelist.net/character/278746/Bok,Bok,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,0.0,Baek's twin sister.\n\nNo voice actors have be...
1327,278696.0,https://myanimelist.net/character/278696/Asuka...,Asuka Nagata,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,0.0,NaN
...,...,...,...,...,...,...,...
209787,282042.0,https://myanimelist.net/character/282042/Alina,Alina,アリナ,https://cdn.myanimelist.net/images/characters/...,0.0,NaN
209788,282042.0,https://myanimelist.net/character/282042/Alina,Alina,アリナ,https://cdn.myanimelist.net/images/characters/...,0.0,NaN
209789,282042.0,https://myanimelist.net/character/282042/Alina,Alina,アリナ,https://cdn.myanimelist.net/images/characters/...,0.0,NaN
209850,282115.0,https://myanimelist.net/character/282115/Cavitt,Cavitt,キャビット,https://cdn.myanimelist.net/images/characters/...,0.0,Lieutenant Rencon's younger sister.\n\nNo voic...


### TENTH DATASET

In [37]:
character_anime_works_df

,anime_mal_id,character_mal_id,character_name,role
0,2928,5781,Atoli,Main
1,2928,33,Haseo,Main
2,2928,32,Ovan,Main
3,2928,34,Shino,Main
4,2928,5785,Aina,Supporting
...,...,...,...,...
236811,31245,137157,"Shibasaki, Ken",Supporting
236812,36305,136064,"Hamanaka, Midori",Main
236813,36305,133916,"Narumi, Sena",Main
236814,36305,124942,"Hayasaka, Akari",Supporting


In [38]:
character_anime_works_df.dtypes

anime_mal_id         int64
character_mal_id     int64
character_name      object
role                object
dtype: object

In [39]:
character_anime_works_df.isna().sum()

anime_mal_id        0
character_mal_id    0
character_name      0
role                0
dtype: int64

In [41]:
character_anime_works_df.loc[character_anime_works_df.duplicated(subset=['anime_mal_id', 'character_mal_id'])]

,anime_mal_id,character_mal_id,character_name,role


There is no need to clean this dataset 

### ELEVENTH DATASET

In [43]:
person_voice_works_df

,person_mal_id,role,anime_mal_id,character_mal_id,language
0,1,Main,55830,2514,Japanese
1,1,Supporting,60602,2822,Japanese
2,1,Supporting,59229,140499,Japanese
3,1,Supporting,60427,275856,Japanese
4,1,Supporting,62067,190335,Japanese
...,...,...,...,...,...
489511,89839,Supporting,40111,274622,Mandarin
489512,89840,Supporting,60544,266412,Mandarin
489513,89841,Supporting,60544,266416,Mandarin
489514,89842,Supporting,36896,279922,Japanese


In [44]:
person_voice_works_df.dtypes

person_mal_id        int64
role                object
anime_mal_id         int64
character_mal_id     int64
language            object
dtype: object

In [51]:
person_voice_works_df.query('language != "Japanese"')

,person_mal_id,role,anime_mal_id,character_mal_id,language
1221,3,Supporting,37491,2944,Italian
1701,5,Supporting,52955,157570,English
3664,10,Supporting,59192,146735,English
3665,10,Main,59226,24734,English
3666,10,Supporting,58567,265497,English
...,...,...,...,...,...
489509,89838,Supporting,60544,266413,Mandarin
489510,89839,Supporting,60544,274622,Mandarin
489511,89839,Supporting,40111,274622,Mandarin
489512,89840,Supporting,60544,266412,Mandarin


In [52]:
person_voice_works_df.isna().sum()

person_mal_id       0
role                0
anime_mal_id        0
character_mal_id    0
language            0
dtype: int64

In [56]:
# check deep the duplicates
person_voice_works_df.loc[person_voice_works_df.duplicated(subset=['person_mal_id', 'role', 'anime_mal_id', 'character_mal_id'], keep='first')]

,person_mal_id,role,anime_mal_id,character_mal_id,language
173916,5742,Main,1011,668,English
173917,5742,Main,1008,668,English
173918,5742,Main,1010,668,English
173919,5742,Main,1007,668,English
173920,5742,Main,792,668,English
...,...,...,...,...,...
478139,77591,Supporting,41025,151299,Portuguese (BR)
478140,77591,Main,431,508,Portuguese (BR)
478141,77591,Main,2202,7952,Portuguese (BR)
478142,77591,Supporting,1069,77729,Portuguese (BR)


### TWELFTH DATASET

In [58]:
# Should we delete the last five columns
profiles_df

,username,gender,birthday,location,joined,watching,completed,on_hold,dropped,plan_to_watch
0,ishikawas,NaN,NaN,South Korea,NaN,NaN,NaN,NaN,NaN,NaN
1,CKK2,NaN,NaN,United States,"Dec 1, 2018",3,182,15,0,405
2,--------788,Female,NaN,Mexico,"Oct 4, 2022",1,64,0,0,1
3,potatoaris,NaN,NaN,Spain,"Oct 2, 2018",5,1,0,0,4
4,Rinrintan,NaN,NaN,Japan,"May 12, 2019",20,311,40,16,34
...,...,...,...,...,...,...,...,...,...,...
337150,ariyanroy04,NaN,NaN,United States,"Mar 30, 2021",35,130,0,0,489
337151,ariyanvk18,Male,NaN,Turkey,"Jan 24, 2022",7,101,1,12,21
337152,ariyoskz,NaN,NaN,Germany,"Jul 7, 2022",1,155,2,11,15
337153,arizima23,Female,"Jun 23, 1998",Spain,"Mar 15, 2019",15,64,0,4,4


In [59]:
profiles_df.dtypes

username         object
gender           object
birthday         object
location         object
joined           object
watching         object
completed        object
on_hold          object
dropped          object
plan_to_watch    object
dtype: object

In [60]:
profiles_df.isna().sum()

username              1
gender           170876
birthday         215826
location              0
joined             1676
watching           1678
completed          1678
on_hold            1678
dropped            1678
plan_to_watch      1678
dtype: int64

In [65]:
profiles_df.query('username.isna()')

,username,gender,birthday,location,joined,watching,completed,on_hold,dropped,plan_to_watch
271344,NaN,NaN,NaN,Mexico,"Jul 28, 2007",2,2,2,0,1


In [67]:
ratings_df.query('username.isna()')

,username,anime_id,status,score,is_rewatching,num_watched_episodes
100051758,NaN,1482,watching,9,0.0,35
100051759,NaN,1735,watching,9,0.0,21
100051760,NaN,121,completed,10,0.0,51
100051761,NaN,136,completed,8,0.0,62
100051762,NaN,269,on_hold,7,0.0,33
100051763,NaN,1818,on_hold,7,0.0,11
100051764,NaN,1535,plan_to_watch,6,0.0,4


In [70]:
profiles_df.loc[profiles_df.duplicated(subset=['username'], keep='first')]

,username,gender,birthday,location,joined,watching,completed,on_hold,dropped,plan_to_watch


### THIRTEENTH DATASET

In [72]:
recommendations_df

,mal_id,recommendation_mal_id
0,3269,317
1,3269,6922
2,3269,299
3,3269,3446
4,3269,5681
...,...,...
105244,31245,1689
105245,31245,35434
105246,31245,31798
105247,31245,21995


In [73]:
recommendations_df.dtypes

mal_id                   int64
recommendation_mal_id    int64
dtype: object

In [74]:
details_df.query('mal_id == 3269')

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,demographics,source,rating,episodes,season,year,producers,explicit_genres,licensors,streaming
3,3269,.hack//G.U. Trilogy,.hack//G.U. Trilogy,https://myanimelist.net/anime/3269/hack__GU_Tr...,https://cdn.myanimelist.net/images/anime/1566/...,Movie,Finished Airing,7.06,15373.0,2007-12-22T00:00:00+00:00,...,[],Game,PG-13 - Teens 13 or older,1.0,NaN,NaN,['Bandai Visual'],[],"['Funimation', 'Bandai Entertainment']",[]


In [76]:
recommendations_df.isna().sum()

mal_id                   0
recommendation_mal_id    0
dtype: int64

In [77]:
recommendations_df.loc[profiles_df.duplicated(keep='first')]

,mal_id,recommendation_mal_id
